In [1]:
import pdblp
from xbbg import blp
import workdays
import datetime
import numpy as np
import pandas as pd

T = ["BPITTO01 Index","SBWGNJYC Index","SBUSJYC Index","TPXDDVD Index","TPXDREIT Index","M0KO Index","SREITJWJ Index"]
T += ["USDJPYCR CMPN Curncy","EURJPYCR CMPN Curncy"]
LS = ["SBJ","SBF","SBU","EQJ","REJ","EQF","REF","USD","EUR"]

con = pdblp.BCon(timeout=10000)
con.start()
d_from = workdays.workday(datetime.datetime.today(), days=-260).strftime("%Y%m%d")
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")
df = con.bdh(T, ["px_last"], d_from, d_to,elms = [("nonTradingDayFillMethod","PREVIOUS_VALUE"),("nonTradingDayFillOption","NON_TRADING_WEEKDAYS")]).reset_index()   #blp.bdhでもok

df = df[["date"]+T]
df.columns = ["Date"] + LS
df.insert(loc = 1, column= "ME", value= 0) #MonthEnd
df

,Date,ME,SBJ,SBF,SBU,EQJ,REJ,EQF,REF,USD,EUR
0,2022-06-16,0,376.131,363.1481,342.1012,3018.24,4358.98,4921.26,1324.15,183.1841,143.4738
1,2022-06-17,0,376.136,364.2143,343.1716,2966.68,4375.11,5025.57,1356.73,187.0928,145.8281
2,2022-06-20,0,375.584,363.2362,343.2015,2939.28,4336.44,5035.24,1358.35,187.1921,146.0592
3,2022-06-21,0,375.368,361.7606,341.5655,2999.49,4359.79,5180.32,1380.11,189.2867,147.9899
4,2022-06-22,0,375.398,365.6740,345.0298,2993.74,4322.67,5156.33,1385.84,188.8592,148.1054
...,...,...,...,...,...,...,...,...,...,...,...
255,2023-06-08,0,375.103,345.8664,326.7251,3639.01,4387.79,6113.85,1406.24,200.6785,157.3040
256,2023-06-09,0,375.351,346.0816,326.0296,3693.51,4438.24,6133.74,1405.80,201.4132,157.3865
257,2023-06-12,0,375.380,346.2371,326.3547,3717.50,4415.85,6189.49,1405.52,201.7579,157.7777
258,2023-06-13,0,375.902,344.8606,324.6872,3760.71,4385.80,6247.39,1411.75,202.6811,159.0054


In [3]:
for i in range(2,len(df)):
    if df.loc[i,"Date"].month != df.loc[i-1,"Date"].month:
        if df.loc[i-1,"Date"].month ==3:
            df.loc[i-1,"ME"] = 5 #Year
        elif df.loc[i-1,"Date"].month ==9:
            df.loc[i-1,"ME"] = 3 #Halfyear
        elif df.loc[i-1,"Date"].month in [6,12]:
            df.loc[i-1,"ME"] = 2 #Quater
        else:
            df.loc[i-1,"ME"] = 1
df.loc[len(df)-1,"ME"] = 5

df = df.loc[df["ME"]>=1,:].reset_index(drop=True)

for i in LS:
    df.loc[i] = df[i].pct_change()

df = df.iloc[1:,:].reset_index(drop=True)


Cor = df[LS].corr()
val, vec = np.linalg.eig(Cor)


vec = pd.DataFrame(vec)
for i in range(len(vec)):
    vec.iloc[i,:] = vec.iloc[i,:].apply("{:.2f}".format)


val /=  sum(val)
val = pd.DataFrame(val,columns=["val"])
for i in range(len(val)):
    val.loc[i,"accumulating"] = sum(val.loc[:i,"val"]) 

for i in range(len(val)):
    val.iloc[i,:] = val.iloc[i,:].apply("{:.2%}".format)
    
print(val)


vec.index = LS
for i in range(len(Cor)):
    Cor.iloc[i,:] = Cor.iloc[i,:].apply("{:.2f}".format)
vec

       val accumulating
0   88.24%       88.24%
1   24.95%      113.19%
2  -13.60%       99.59%
3    0.33%       99.93%
4    0.10%      100.02%
5   -0.04%       99.98%
6    0.02%      100.00%
7   -0.00%      100.00%
8    0.00%      100.00%


,0,1,2,3,4,5,6,7,8
SBJ,0.36,0.02,-0.04,0.08,0.18,-0.63,0.49,-0.35,0.26
SBF,0.35,-0.27,0.35,-0.13,0.16,0.12,0.10,0.63,0.47
SBU,0.35,-0.26,0.34,-0.14,0.22,0.10,0.21,-0.07,-0.75
EQJ,0.35,0.27,-0.37,-0.52,0.29,0.45,-0.01,-0.27,0.17
REJ,0.35,-0.16,0.20,0.13,0.12,-0.18,-0.81,-0.29,0.10
EQF,0.36,0.27,-0.38,-0.21,-0.29,-0.42,-0.19,0.48,-0.29
REF,0.35,-0.11,0.13,0.00,-0.83,0.25,0.14,-0.26,0.11
USD,0.35,0.18,-0.26,0.79,0.15,0.32,0.08,0.13,-0.07
EUR,0.02,0.80,0.60,-0.00,-0.00,0.00,0.00,-0.00,0.00
